### Custom Exceptions

In [1]:
class TimeoutError(Exception):
    """Timeout exception"""

In [3]:
try:
    raise TimeoutError('timeout error')
except TimeoutError as ex:
    print(ex)

timeout error


In [5]:
import sys

try:
    raise TimeoutError('timeout error')
except:
    ex_type, ex, tb = sys.exc_info()
    print(ex_type, ex, tb)

<class '__main__.TimeoutError'> timeout error <traceback object at 0x7f7ecc06c980>


In [6]:
try:
    raise TimeoutError('timeout exception')
except Exception as ex:
    print(ex.args, ex.__traceback__)

('timeout exception',) <traceback object at 0x7f7ebd76e8c0>


In [7]:
class ReadOnlyError(AttributeError):
    """Indicates an attribute is rean-only"""

In [ ]:
try:
    raise ReadOnlyError('Account number is read-only', 'BA10001')
except ReadOnlyError as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-only', 'BA10001')


In [10]:
try:
    raise ReadOnlyError('Account number is read-only', 'BA10001')
except BaseException as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-only', 'BA10001')


In [14]:
class WebScraperException(Exception):
    """Base exception for WebScraper"""

class HTTPException(WebScraperException):
    """General HTTP exception for WebScraper"""

class InvalidUrlException(HTTPException):
    """Indicates the url is invalid (dns lookup fail)"""

class TimeoutException(HTTPException):
    """Indicates a general timeout exception in http connectivity"""

class PingTimeoutException(TimeoutException):
    """Ping time out"""

class LoadTimeoutEception(TimeoutException):
    """Page load timeout"""

class ParserException(WebScraperException):
    """Geneeral page parsing exception"""

```
WebScraperException
    - HTTPException
        - InvalidUrlException
        - TimeoutException
            - PingTimeoutException
            - LoadTimeoutException
    - ParserException
```

In [16]:
try:
    raise PingTimeoutException('Ping to www ... timed out')
except WebScraperException as ex:
    print(repr(ex))

PingTimeoutException('Ping to www ... timed out')


In [17]:
class APIException(Exception):
    """Base API Exception"""

In [19]:
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""

class DBException(ApplicationException):
    """General database exception"""

class DBConnectionError(DBException):
    """Indicates an error connecting to database"""

class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""

class NotFoundError(ClientException):
    """Indicates resource was not found"""

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""

So we have this exception hierarchy:

```
APIException
   - ApplicationException (5xx errors)
       - DBException
           - DBConnectionError
   - ClientException
       - NotFoundError
       - NotAuthorizedError
```

In [20]:
class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type

In [21]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f'Account number {account_id} is invalid.')

    if account_id < 100:
        raise DBConnectionError('Permanent failure connecting to database')
    elif account_id < 200:
        raise NotAuthorizedError('User does not have permissions to read this account.')
    elif account_id < 300:
        raise NotFoundError('Account not found')
    else:
        return Account(account_id, 'Savings')

In [22]:
from http import HTTPStatus

def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except ApplicationException as ex:
        return HTTPStatus.INTERNAL_SERVER_ERROR, str(ex)
    except NotFoundError as ex:
        return HTTPStatus.NOT_FOUND, 'The account {} does not exist'.format(account_id)
    except NotAuthorizedError as ex:
        return HTTPStatus.UNAUTHORIZED, 'You do not have the proper authorizatoin'
    except ClientException as ex:
        return HTTPStatus.BAD_REQUEST, str(ex)
    else:
        return HTTPStatus.OK, {"id": account.account_id, "type": account.account_type}


In [24]:
get_account('abc')

(<HTTPStatus.BAD_REQUEST: 400>, 'Account number abc is invalid.')

In [25]:
get_account(50)

(<HTTPStatus.INTERNAL_SERVER_ERROR: 500>,
 'Permanent failure connecting to database')

In [26]:
get_account(150)

(<HTTPStatus.UNAUTHORIZED: 401>, 'You do not have the proper authorizatoin')

In [27]:
get_account(250)

(<HTTPStatus.NOT_FOUND: 404>, 'The account 250 does not exist')

In [28]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})

In [29]:
class APIException(Exception):
    """Base API Exception"""

    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'We are sorry! An unexpected error occurred on our end.'

    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)

        if user_err_msg is not None:
            self.user_err_msg = user_err_msg

In [30]:
try:
    raise APIException()
except APIException as ex:
    print(repr(ex))
    print(ex.user_err_msg)

APIException('API exception occurred')
We are sorry! An unexpected error occurred on our end.


In [32]:
try:
    raise APIException('custom message...', 10, 20, user_err_msg='custom user message...')
except APIException as ex:
    print(repr(ex))
    print(ex.user_err_msg)

APIException('custom message...', 10, 20)
custom user message...


In [33]:
import json

In [34]:
class APIException(Exception):
    """Base API Exception"""

    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'We are sorry! An unexpected error occurred on our end.'

    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)

        if user_err_msg is not None:
            self.user_err_msg = user_err_msg

    def to_json(self):
        err_object = {'status': self.http_status, 'message': self.user_err_msg}
        return json.dumps(err_object)

In [35]:
try:
    raise APIException()
except APIException as ex:
    print(repr(ex))
    print(ex.to_json())

APIException('API exception occurred')
{"status": 500, "message": "We are sorry! An unexpected error occurred on our end."}


In [3]:
from http import HTTPStatus
import json
from datetime import datetime

In [4]:
class APIException(Exception):
    """Base API Exception"""

    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred'
    user_err_msg = 'We are sorry! An unexpected error occurred on our end.'

    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)

        if user_err_msg is not None:
            self.user_err_msg = user_err_msg

    def to_json(self):
        err_object = {'status': self.http_status, 'message': self.user_err_msg}
        return json.dumps(err_object)

    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "http_status": self.http_status,
            "message": self.args[0] if self.args else self.internal_err_msg,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.now().isoformat()}: {exception}')


In [5]:
try:
    raise APIException()
except APIException as ex:
    ex.log_exception()
    print(ex.to_json)

EXCEPTION: 2026-02-03T12:07:47.086671: {'type': 'APIException', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'API exception occurred', 'args': ()}
<bound method APIException.to_json of APIException('API exception occurred')>


In [6]:
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'Generic server side exception.'
    user_err_msg = 'We are sorry. An unexpected error occurred on our end.'

class DBException(ApplicationException):
    """General database exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'Database exception'
    user_err_msg = 'We are sorry. An unexpected error occurred on our end.'

class DBConnectionError(DBException):
    """Indicates an error connecting to database"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'Database exception'
    user_err_msg = 'We are sorry. An unexpected error occurred on our end.'

class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = 'Client submitted bad request'
    user_err_msg = 'A bad request was received.'

class NotFoundError(ClientException):
    """Indicates resource was not found"""
    http_status = HTTPStatus.NOT_FOUND
    internal_err_msg = 'Resource not found'
    user_err_msg = 'Requested resource was not found.'

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""
    http_status = HTTPStatus.UNAUTHORIZED
    internal_err_msg = 'Client not authorized to perform operstion'
    user_err_msg = 'You are not authorized to perform this request.'

In [17]:
class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type

In [18]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f'Account number {account_id} is invalid.',
                              f'account_id = {account_id}',
                              'type error - account number not an integer')

    if account_id < 100:
        raise DBConnectionError('Permanent failure connecting to database',
                                'db=db01')

    elif account_id < 200:
        raise NotAuthorizedError('User does not have permissions to read this account.',
                                 f'account_id={account_id}')

    elif account_id < 300:
        raise NotFoundError('Account not found',
                            f'account_id={account_id}')

    else:
        return Account(account_id, 'Savings')

In [8]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except APIException as ex:
        ex.log_exception()
        return ex.to_json()
    else:
        return HTTPStatus.OK, {"id": account.account_id, "type": account.account_type}

In [9]:
get_account('ABC')

EXCEPTION: 2026-02-03T12:24:18.422844: {'type': 'ClientException', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'message': 'Account number ABC is invalid.', 'args': ('account_id = ABC', 'type error - account number not an integer')}


'{"status": 400, "message": "A bad request was received."}'

In [13]:
get_account(50)

EXCEPTION: 2026-02-03T12:25:06.217802: {'type': 'DBConnectionError', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'Permanent failure connecting to database', 'args': ('db=db01',)}


'{"status": 500, "message": "We are sorry. An unexpected error occurred on our end."}'

In [14]:
get_account(150)

EXCEPTION: 2026-02-03T12:25:14.702338: {'type': 'NotAuthorizedError', 'http_status': <HTTPStatus.UNAUTHORIZED: 401>, 'message': 'User does not have permissions to read this account.', 'args': ('account_id=150',)}


'{"status": 401, "message": "You are not authorized to perform this request."}'

In [15]:
get_account(250)

EXCEPTION: 2026-02-03T12:25:19.067443: {'type': 'NotFoundError', 'http_status': <HTTPStatus.NOT_FOUND: 404>, 'message': 'Account not found', 'args': ('account_id=250',)}


'{"status": 404, "message": "Requested resource was not found."}'

In [19]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})

In [24]:
class AppException(Exception):
    """Generic application exception"""

class NegativeIntegerError(AppException, ValueError):
    """Used to indicate an error when an integer is negative"""

In [25]:
ex = NegativeIntegerError()

In [27]:
isinstance(ex, AppException), isinstance(ex, ValueError)

(True, True)

In [21]:
def set_age(age):
    if age < 0:
        raise NegativeIntegerError('age cannot be negative')

In [22]:
try:
    set_age(-10)
except NegativeIntegerError as ex:
    print(repr(ex))


NegativeIntegerError('age cannot be negative')


In [29]:
try:
    set_age(-10)
except ValueError as ex:
    print(repr(ex))

NegativeIntegerError('age cannot be negative')


In [30]:
try:
    set_age(-10)
except AppException as ex:
    print(repr(ex))

NegativeIntegerError('age cannot be negative')
